### Блок теоретических вопросов

*Основное отличие DBSCAN и K-Means*: 

1.	Благодаря своей простой структуре K-Means обучается быстрее, чем DBSCAN
2.	DBSCAN  способен формировать кластеры любой формы, в то время как K-Means только определенной формы 
3.	K-Means можно использовать для генерации новых признаков, а DBSCAN для этого неприменим







**Ответ: 2)** Методу DBSCAN важно, чтобы точки находились плотно друг к другу, а их форма и расположение центра не особо важно.

___________________________________________

*В чем состоит сложность при работе с DBSCAN*: 

1.	Сложен в обучении
2.	Необходимо делать предположения о следующих двух параметрах: размер окрестности и количество соседей.
3.	Не учитывает, что в данных могут быть выбросы




**Ответ: 2)** По построению DBSCAN, в дейтсвительно, предполагает некоторые априорные представления об этих двух параметрах

___________________________________________

*Какая задача напрямую не решается кластеризацией?*: 

1.	Поиск похожих клиентов для рекомендательных систем
2.	Предсказание ВВП по заголовкам новостных источников 
3.	Выделение групп похожих ответов в соц. опросах
4.	Объединение сообществ в соц. сетях по тематикам





**Ответ: 2)** Предсказание ВВП - это задача обучения с учителем. Конечно, решая ее, можно использовать алгоритмы кластеризации, скажем, для создания новых категориальных признаков. Но напрямую кластеризировать ВВП не получится :)

___________________________________________

### Блок практики

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = 500

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

# Кластеризация

Наша задача - предсказать есть диабет у индивида или нет. В качестве таргета - колонка Diabetes. В нем три различных значения: `0`, `1`, `2`. `0` означает, что наблюдаемой здоров, `1` значит, что есть риск диабета, `2` означает наличие диабета. В качестве признаков будем использовать пол, количество лет в США, доход семьи и некоторые показатели, измеренные медицинскими работниками.  

**Задание 1.** В этой части ДЗ попробуем использовать кластеризацию как инструмент при проведении моделирования в задаче классификации

In [ ]:
### Загрузим датасет

df = pd.read_csv('datahw21.csv', index_col='Unnamed: 0')

df.head()

In [ ]:
### Посмотрим как устроены данные
### Изобразим корреляционную матрицу

colormap = plt.cm.viridis
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), linewidths=0.1, vmax=1.0, square=True, cmap=colormap, annot=True)

In [ ]:
### Разделим выборку на трейн-тест

data = df.drop(['Diabetes'], axis=1)
target = df[['Diabetes']]

X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target, 
                                                    test_size=0.25,
                                                    random_state=1)

Для того, чтобы использовать K-means, лучше будет отнормировать данные. 

In [ ]:
### Воспользуемся StandardScaler

cols = X_train.columns
scaler = StandardScaler()
X_train_sc = pd.DataFrame(scaler.fit_transform(X_train), columns=cols)
X_test_sc = pd.DataFrame(scaler.transform(X_test), columns=cols)

Обучим K-Means с параметрами `n_clusters` = 3, `tol` = 0.0005. Выбор параметров обусловлен тем, что у нас три возможных значения таргета. Но в целом основной подход подбора количества кластеров - по кривой зависимости внутрикластерного и межкластерного расстояний от количества кластеров.

Установите `random_state` = 1

In [ ]:
kms = KMeans(n_clusters = 3, 
             tol = 0.0005,
             random_state=1)

kms.fit_predict(X_train_sc)

print ("parameters: ", kms.get_params)
print ("preict: ", kms.predict)
print ("\nscore: %.2f" % kms.score(X_test_sc))

Посчитаем качество на изначальных данных(нормированных). Для этого обучите с дефолтными параметрами `RandomForestClassifier`, `LogisticRegression`, `LinearSVC`. Там, где нужно, установите `random_state` = 1. (1б)

In [ ]:

rf = RandomForestClassifier(random_state=1)
lr = LogisticRegression()
svm = LinearSVC()

rf.fit(X_train_sc, y_train)
lr.fit(X_train_sc, y_train)
svm.fit(X_train_sc, y_train)

print('RF acc:', accuracy_score(y_test, rf.predict(X_test_sc)))
print('LR acc:',accuracy_score(y_test, lr.predict(X_test_sc)))
print('SVM acc:',accuracy_score(y_test, svm.predict(X_test_sc)))

Добавьте в признаковое описание номер кластера и посчитайте качество с новым признаком! Стало ли качество хоть сколько-то лучше? (1б)

In [ ]:
X_train_k = pd.concat(
    [
        X_train_sc,
        pd.DataFrame(kms.predict(X_train_sc), columns=['Cluster_num'])   # <-- добавляем кластер как признак
    ],
    axis=1
)
X_test_k = pd.concat(
    [X_test_sc,
     pd.DataFrame(kms.predict(X_test_sc), columns=['Cluster_num'])       # <-- добавляем кластер как признак
    ],
    axis=1
)

rf = RandomForestClassifier(random_state=1)
lr = LogisticRegression()
svm = LinearSVC()

rf.fit(X_train_k, y_train)
lr.fit(X_train_k, y_train)
svm.fit(X_train_k, y_train)

print('RF acc:', accuracy_score(y_test, rf.predict(X_test_k)))
print('LR acc:',accuracy_score(y_test, lr.predict(X_test_k)))
print('SVM acc:',accuracy_score(y_test, svm.predict(X_test_k)))

Посчитаем расстояния от объектов до центров кластеров. Для этого воспользуемся методом `transform` обученного класса kmeans.

Обучим и посчитаем метрики исключительно на расстояниях до центра. Убедимся, что такой подход имеет право на существование, если данные позволяют, то качество не сильно должно пострадать. А в каких-то случаях может оказаться даже лучше! Таким образом можно снижать размерность данных. (2б)

In [ ]:
new1 = kms.transform(X_train_sc)            #  <--- расстояния от объектов до центра кластеров на трейне
new2 = kms.transform(X_test_sc)             #  <--- расстояния от объектов до центра кластеров на трейне

rf = RandomForestClassifier(random_state=1)
lr = LogisticRegression()
svm = LinearSVC()

rf.fit(new1, y_train)
lr.fit(new1, y_train)
svm.fit(new1, y_train)

print('RF acc:', accuracy_score(y_test, rf.predict(new2)))
print('LR acc:',accuracy_score(y_test, lr.predict(new2)))
print('SVM acc:',accuracy_score(y_test, svm.predict(new2)))

**Задание 2 (Бонус)** Задача кластеризации может использоваться не только для специфических задач группировки данных, но и для оптимизации других методов. Вы уже знаете, что одна из основных проблем kNN в скорости его предсказания. В этом задании попробуем ускорить работу kNN с помощью кластеризации, не теряя при этом сильно в качестве.

Сначала загрузим уже известные вам данные клиентов страховой компании.

In [ ]:
### Загрузим известный нам датасет

data = pd.read_csv('processed_vehicle_inssurance.csv')
data.head()

In [ ]:
### Разделим выборку на трейн-тест

from sklearn.model_selection import train_test_split
X = data.drop('Response', axis=1)[:25000]
y = data['Response'][:25000]

X_train, X_test, y_train, y_test = train_test_split(X, y.values,
                                                    random_state=0,
                                                    test_size=0.2)

X_train.shape, X_test.shape

In [ ]:
### Нормируем данные

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Обучите обычный kNN с одним соседом и измерьте качество, например, взвешенную f-меру, чтобы потом сранить с нашей реализацией. (1б)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

In [ ]:
%%time
y_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, f1_score, roc_curve, auc
print(f1_score(y_test, y_pred, average='weighted'))

Идея ускорения kNN заключается в том, чтобы разбить признаковое пространство (то есть столбцы, а не объекты-строки!) на несколько блоков и кластеризовать каждый блок по-отдельности.

Далее нужно будет заменить в обучающей выборке объекты на их некоторое представление, а именно вместо каждого блока вставить центр кластера, к которому принадлежит эта часть исходного объекта.

Затем подсчет расстояний на этапе применения производится по блокам. Главный плюс алгоритма в том, что у нас уже будут иметься предподсчитанные расстояния по блокам до всех центров кластеров.

Этот алгоритм называется **Product Quantization**. 


Сам алгоритм:

1. Разделите обучающую и контрольную выборки на блоки: равномерно по индексам столбцов.


2. На каждом блоке обучите K-Means и примените transform к соотв. блоку контрольной выборки.


3. Посчитайте расстояния от каждого обучающего объекта до каждого объекта из контрольной выборки
(это вы должны сделать, используя матрицы из предыдущего пункта)


4. Определите для каждого тестового объекта k ближайших

В нашей реализации будем использовать следующие значения параметров: \
`m_blocks` = 5 \
`n_clusters` = 100 \
`k` = 1

In [ ]:
def product_quantization(X_train, X_test, m_blocks=5, n_clusters=100):
    dist_table = np.zeros([X_test.shape[0], n_clusters, m_blocks])
    X_train_clusters = np.zeros([X_train.shape[0], m_blocks])
    
    for i in range(m_blocks):
        ### Вырежьте блок из обучающей и контрольной выборок
        block_size = X_test.shape[1] // m_blocks
        X_train_block = X_train[:, i*block_size:(i+1)*block_size].copy()
        X_test_block = X_test[:, i*block_size:(i+1)*block_size].copy()
        
        ### Обучите K-Means и примените transform на тестовой выборке
        ### Положите посчитанные расстояния до центров в матрицу dist_table
        kmeans = KMeans(n_clusters=n_clusters).fit(X_train_block)
        dist_table[:, :, i] = kmeans.transform(X_test_block)
        
        ### Положите метки кластеров в матрицу X_train_clusters
        X_train_clusters[:, i] = kmeans.predict(X_train_block)
        
    return X_train_clusters, dist_table

**Совет на будущее в практике:** Обучайте все на подвыборке данных, так как итоговая матрица при подсчете kNN будет занимать очень много оперативной памяти. 


In [ ]:
%%time
X_train_clusters, dist_table = product_quantization(X_train, X_test, 
                                                    m_blocks=5, 
                                                    n_clusters=100)

Теперь с помощью полученных таблиц осталось посчитать расстояния до каждого объекта обучающей выборки. 
1. Сначала возведите в квадрат `dist_table`, чтобы получить сумму квадратов, а не l2-норму.

2. Для каждого блока по предсказанным меткам класса в `X_train_clusters` отберите соответсвтующие расстояния из dist_table

3. Просуммируйте квадраты расстояний по всем блокам.

4. Найдите индексы самых маленький расстояний и по ним выберите объекты из y_train, это и будут наши предсказания

Замерьте качество, как изменилась скорость работы? 

In [ ]:
sumsq_table = dist_table ** 2
sumsq_table.shape

X_train_clusters = X_train_clusters.astype('int')

In [ ]:
%%time
distances = np.zeros([sumsq_table.shape[0], X_train.shape[0]])
m_blocks = 5

for b in range(m_blocks):
    distances += sumsq_table[:, X_train_clusters[:, b], b]

y_pred = y_train[np.argmin(distances, axis=1)]

Наша реализация на python работает медленнее по сравнению с библиотечным kNN. Однако реализация на более низкоуровневых языках программирования (C++) и на большом количестве данных данный метод на самом деле позволяет ускорять подсчет расстояний.

In [ ]:
print(f1_score(y_test, y_pred, average='weighted'))